In [ ]:
# Import the necessary libraries and functions
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import pandas as pd
import csv

In [127]:
# Be sure to download the Chrome driver version that is appropriate for your browser. 
# Substitute the service item with the location of where your driver is saved.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")

# Opening the website in the Chrome driver
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://kdocrepository.doc.ks.gov/kasper/')
ids = []


In [ ]:
driver.find_element("xpath", ".//*[@value='Agree']").click()
driver.find_element("xpath", ".//*[@href='#collapse_advanced']").click()
page_source = driver.page_source

# The BeautifulSoup library is useful for scraping static pages. We are using it here to scrape for the facility names
soup = BeautifulSoup(page_source, 'lxml')
select = soup.find('select', {'name': 'facility'})
options = select.find_all('option')
option_text = [option.text for option in options]
prisons = option_text[1:]
prisons = [s.rstrip() if s.endswith(' ') else s for s in prisons]

# Return back to the main page with facility names stored
driver.get('https://kdocrepository.doc.ks.gov/kasper/')

In [ ]:
for prison in prisons:
    driver.find_element("xpath", ".//*[@href='#collapse_advanced']").click()
    supervision = Select(driver.find_element("name", 'supervisionType'))
    supervision.select_by_visible_text("Inmate")
    facility = Select(driver.find_element("name", 'facility'))
    facility.select_by_visible_text(prison)
    driver.find_element("name", "submitBtn").click()

    driver.implicitly_wait(10)

    while True:
        try:
            next = driver.find_element("xpath", "//a[contains(text(), 'Next')]")
        except:
            break
        table = driver.find_element("xpath", "//table")
        rows = table.find_elements("xpath", ".//tr")

        for row in rows:
            cells = row.find_elements("tag name", "td")
            if cells:
                ids.append(cells[0].text)
        next.click()
        
    driver.get('https://kdocrepository.doc.ks.gov/kasper/')

In [128]:
with open('/Users/tumendemberelshalkhaan/Desktop/Kansas_DOC.csv', newline='') as ids:
    # Create a CSV reader object
    csvreader = csv.reader(ids)
    
    # Convert the CSV reader object to a list
    ids = [item for sublist in csvreader for item in sublist]

In [125]:
ks = pd.DataFrame(columns=['ID', 'Name', 'Race', 'Date Of Birth', 'Facility', 'County', 'Offense Description', 'Offense Date', 'Offense Code', 'Sentence Length', 'Sentencing Date', 'Admission Date', 'Release Date'])

In [129]:
driver.find_element("xpath", ".//*[@value='Agree']").click()
driver.find_element("xpath", ".//*[@href='#collapse_advanced']").click()


In [138]:
driver.find_element("xpath", ".//*[@href='#collapse_advanced']").click()
supervision = Select(driver.find_element("name", 'supervisionType'))
supervision.select_by_visible_text("Inmate")
facility = Select(driver.find_element("name", 'facility'))
facility.select_by_visible_text("El Dorado CF-Central")
driver.find_element("name", "submitBtn").click()
driver.implicitly_wait(10)
driver.find_element("xpath", ".//*[@href='/kasper/search/detail?kdocNumber=54639']").click()

In [139]:
for id in ids[6601:]:
    driver.get('https://kdocrepository.doc.ks.gov/kasper/search/detail?kdocNumber=' + id)

    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'lxml')

    try:
        release_date = soup.find('span', text=('Earliest Possible Release Date (1)')).next_sibling.get_text()
    except AttributeError:
        release_date = None
    try:
        admin_date = soup.find('span', text=('Admission Date')).next_sibling.get_text()
    except AttributeError:
        admin_date = None
    try:
        fac = soup.find('a', href=lambda href: href and href.startswith('http://www.doc.ks.gov/facilities/')).get_text()
    except AttributeError:
        fac = None

    name_table = soup.find(id = 'names')
    name = name_table.find('td', string = 'Conviction').find_next_sibling('td').get_text()

    birthdate_table = soup.find(id = 'birthdates')
    dob = birthdate_table.find('td', string = 'True').find_next_sibling('td').get_text()
    age = birthdate_table.find('td', string = 'True').find_next_sibling('td').find_next_sibling('td').get_text()

    demo_table = soup.find(id = 'demographics')
    demos = demo_table.find_all('td')
    gender = demos[4].get_text()
    race = demos[5].get_text()

    con_table = soup.find(id = 'convictions').find('tbody')

    offenses = []
    offdates = []
    sentences = []
    sendates = []
    counties = []
    statuses = []

    for row in con_table.find_all('tr'):
        data = [cell.get_text(strip=True) for cell in row.find_all('td')]
        counties.append(data[0])
        offdates.append(data[2])
        sendates.append(data[3])
        sentences.append(data[4])
        offenses.append(data[5])

    histories = [list(history) for history in zip(offenses, sentences, counties, offdates, sendates)]

    rows = []

    for history in histories:
            
        row = {
            'ID': id,
            'Name': name,
            'Date Of Birth': dob,
            'Gender': gender,
            'Race': race,
            'Facility': fac,
            'Offense Description': history[0],
            'Offense Date': history[3],
            'Sentence Length': history[1],
            'Sentence Date': history[4],
            'County': history[2],
            'Admission Date': admin_date,
            'Release Date': release_date
            }
        rows.append(row)
            
    ks = pd.concat([ks, pd.DataFrame(rows)])

In [140]:
ks.to_csv('/Users/tumendemberelshalkhaan/Desktop/Kansas_Raw.csv', index=False)